##### Copyright 2018 The TensorFlow Hub Authors.

Licensed under the Apache License, Version 2.0 (the "License");

In [1]:
from absl import logging

import matplotlib.pyplot as plt
import numpy as np
from PIL import Image, ImageOps
from scipy.spatial import cKDTree
from skimage.feature import plot_matches
from skimage.measure import ransac
from skimage.transform import AffineTransform
from six import BytesIO
import time
from sklearn.metrics.pairwise import cosine_similarity
import glob
import tensorflow as tf
from multiprocessing import Pool
import tensorflow_hub as hub
from six.moves.urllib.request import urlopen
from numba import jit
from collections import defaultdict
import delf_compare_images_multiprocessing
import json

In [2]:
# gpu = tf.config.experimental.list_physical_devices('GPU')
# print(gpu)
# if gpu:
#     try:
#       # Currently, memory growth needs to be the same across GPUs
# #         tf.config.experimental.set_memory_growth(gpu[0], True)
#         tf.config.experimental.set_virtual_device_configuration(
#         gpu[0],
#         [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=512)])
#         print(len(gpu), "Physical GPUs")
#     except RuntimeError as e:
#       # Memory growth must be set before GPUs have been initialized
#       print(e)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
1 Physical GPUs


In the next cell, we specify the URLs of two images we would like to process with DELF in order to match and compare them.

In [2]:
delf = hub.load('https://tfhub.dev/google/delf/1').signatures['default']

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [6]:
import json
with open('imageIds_landmarkIds_moreThan5_dict.json') as json_file:
    imageIds_landmarkIds_moreThan5_dict = json.load(json_file)
print(type(imageIds_landmarkIds_moreThan5_dict))
print(len(imageIds_landmarkIds_moreThan5_dict))
print(imageIds_landmarkIds_moreThan5_dict[:5])

<class 'dict'>
3909773


TypeError: unhashable type: 'slice'

In [3]:
with open('landmark_ids_greater_than_5_list.json') as json_file:
    landmark_ids_greater_than_5_list = json.load(json_file)
type(landmark_ids_greater_than_5_list)
print(len(landmark_ids_greater_than_5_list))
print(landmark_ids_greater_than_5_list[:5])

125729
[177870, 176528, 192931, 126637, 83144]


In [6]:
# creating dictionary of landmark_ids and delf features for all landmarks that has >5 images to later use to compare features
landmark_ids_images_delf_features_dict = {}
    
# Create a dict of images/files, max_inliers to filter out after DELF
images_to_filter_out_after_delf = {}

# Create a dictionary {'landmark_id' : (tuples with image1 path, image2path, inliers). 
# we will use this list of tuples later to only use the top 100 images with the most similar inliers within each class for modeling
top_pairs_landmark_id_dict = {}
tuple_list = []

In [6]:
# image1_path = "datasets\\set_256\\train\\138982\\ae2cf856f1c12d39.jpg"
# image2_path = "datasets\\set_256\\train\\138982\\e0607570dfd71d89.jpg"
# image3_path = "datasets\\set_256\\train\\138982\\f5fdae36a2d2b558.jpg"
# image4_path = "datasets\\set_256\\train\\138982\\4f647425aec45ba3.jpg"
# image5_path = "datasets\\set_256\\train\\138982\\8da40f7bdb25c6b4.jpg"
# image6_path = "datasets\\set_256\\train\\138982\\9aa65a6851930696.jpg"

# Original Raw Images without resizing
# image1_path = "datasets\\raw_data\\train\\0\\09ed156ffac3e70c.jpg"
# image2_path = "datasets\\raw_data\\train\\0\\26e829f30f657d0c.jpg"
# image3_path = "datasets\\raw_data\\train\\0\\0052fab157ef0378.jpg"
# image4_path = "datasets\\raw_data\\train\\0\\62b7acc90ded9ce5.jpg"
# image5_path = "datasets\\raw_data\\train\\0\\0ddc92c6353c019a.jpg"
# image6_path = "datasets\\raw_data\\train\\226\\b5d8eb502833a58c.jpg"


# images_paths = [image1_path, image2_path, image3_path, image4_path, image5_path, image6_path]


landmark_id = '168098'
images_paths = glob.glob("datasets\\raw_data\\train\\{}\\*.jpg".format(landmark_id))
print("Number of images in the landmark_id, "+ landmark_id + " : ", len(images_paths))
print(images_paths[:2])

Number of images in the landmark_id, 168098 :  500
['datasets\\raw_data\\train\\168098\\0019a9bd5f496ab9.jpg', 'datasets\\raw_data\\train\\168098\\00d07d7fcb35525f.jpg']


Download, resize, save and display the images.

In [7]:
def download_and_resize(path, new_width = 256, new_height = 256):
    image = Image.open(path)
    image = ImageOps.fit(image, (new_width, new_height), Image.ANTIALIAS)
    return image

In [8]:
images = []

for i in range (0,len(images_paths)):
    images.append(download_and_resize(images_paths[i]))
#     plt.subplot(2,3,i+1)
#     plt.imshow(images[i])

In [9]:
def run_delf(image):
    np_image = np.array(image)
    float_image = tf.image.convert_image_dtype(np_image, tf.float32)
    return delf(
        image=float_image,
        score_threshold=tf.constant(100.0),
        image_scales=tf.constant([0.25, 0.3536, 0.5, 0.7071, 1.0, 1.4142, 2.0]),
        max_feature_num=tf.constant(1000))

In [10]:
def get_deep_local_features(images):
    run_delf_image_list_results = []
    start_time = time.time()
    for i in range (0,len(images)):
        run_delf_image_list_results.append(run_delf(images[i]))
    print("Total time to run delf for " + str(len(images)) + " images is : ", time.time() - start_time)
    return run_delf_image_list_results


In [11]:
# results = get_deep_local_features(images)

In [12]:
image_counts = len(images)
loop_iterations = image_counts/100
print(loop_iterations)
if int(loop_iterations) < loop_iterations:
    loop_iterations = int(loop_iterations) + 1
else:
    loop_iterations = int(loop_iterations)
print(loop_iterations)

5.0
5


In [13]:
# numbers = [k for k in range(image_counts)]
# numbers

In [14]:
# start = 0
# step = 100
# for i in range(1,loop_iterations+1):
#     end = step*i
#     if i==loop_iterations:
#         print(numbers[start:len(numbers)])
#         print("here2")
#     else:
#         print(numbers[start:end])
#         start = end
#         print("here3")

In [15]:
start = 0
step = 100
results = []
for i in range(1,loop_iterations+1):
    end = step*i
    if i==loop_iterations:
        results.extend(get_deep_local_features(images[start:len(images)]))
        print("here2")
    else:
        results.extend(get_deep_local_features(images[start:end]))
        start = end
        print("here3")

Total time to run delf for 100 images is :  7.397167205810547
here3
Total time to run delf for 100 images is :  2.313814640045166
here3
Total time to run delf for 100 images is :  2.3466975688934326
here3
Total time to run delf for 100 images is :  2.316619873046875
here3
Total time to run delf for 100 images is :  2.348904848098755
here2


In [17]:
results = get_deep_local_features(images)

Total time to run delf for 500 images is :  11.671324253082275


In [17]:
print(len(results))
print(results[0].keys())
print(type(results))
print(type(results[0]))

26
dict_keys(['scores', 'features', 'scales', 'locations', 'boxes', 'attention', 'descriptors'])
<class 'list'>
<class 'dict'>


In [18]:
for i in range(0,len(images)):
    print(results[i]['descriptors'].shape)

(170, 40)
(143, 40)
(140, 40)
(9, 40)
(131, 40)
(57, 40)
(159, 40)
(118, 40)
(200, 40)
(186, 40)
(165, 40)
(149, 40)
(280, 40)
(141, 40)
(210, 40)
(161, 40)
(133, 40)
(144, 40)
(178, 40)
(108, 40)
(140, 40)
(217, 40)
(175, 40)
(119, 40)
(134, 40)
(132, 40)


In [21]:
print(results)

[{'scores': <tf.Tensor: shape=(170, 1), dtype=float32, numpy=
array([[741.3327  ],
       [729.7612  ],
       [424.1613  ],
       [357.76056 ],
       [346.05573 ],
       [334.17535 ],
       [313.65558 ],
       [287.27386 ],
       [281.10425 ],
       [279.0573  ],
       [274.4751  ],
       [266.76648 ],
       [254.46635 ],
       [254.29897 ],
       [246.76752 ],
       [241.94612 ],
       [223.1681  ],
       [220.96176 ],
       [218.55295 ],
       [218.29485 ],
       [214.4643  ],
       [214.1493  ],
       [213.92245 ],
       [212.62129 ],
       [211.6503  ],
       [210.90652 ],
       [208.39883 ],
       [201.90482 ],
       [200.16292 ],
       [199.06801 ],
       [198.58582 ],
       [196.78947 ],
       [194.5982  ],
       [193.29088 ],
       [193.14145 ],
       [192.84541 ],
       [187.31398 ],
       [187.07195 ],
       [183.73055 ],
       [180.65388 ],
       [179.54124 ],
       [179.12657 ],
       [178.80962 ],
       [177.98195 ],
       [177.45

## Use the locations and description vectors to match the images

In [ ]:
# #@title TensorFlow is not needed for this post-processing and visualization
# def match_images(image1, image2, result1, result2, i, j):
#     distance_threshold = 0.8

#     # Read features.
#     num_features_1 = result1['locations'].shape[0]
#     print("Loaded image %d 's %d features" % (i, num_features_1))

#     num_features_2 = result2['locations'].shape[0]
#     print("Loaded image %d 's %d features" % (j, num_features_2))

#     # Find nearest-neighbor matches using a KD tree.
#     d1_tree = cKDTree(result1['descriptors'])
#     _, indices = d1_tree.query(
#         result2['descriptors'],
#         distance_upper_bound=distance_threshold)
# #     print(indices)
#     print("Shape of all indices retrieved from cKDTree : ",indices.shape)
#     i_set = list(set(indices))
#     i_set.sort()
#     print("Common descriptors set of indices: ", i_set)
#     print("Length of common descriptors : ", len(set(indices)))

#     # Select feature locations for putative matches.
#     locations_2_to_use = np.array([
#         result2['locations'][i,]
#         for i in range(num_features_2)
#         if indices[i] != num_features_1
#     ])
    
#     locations_1_to_use = np.array([
#         result1['locations'][indices[i],]
#         for i in range(num_features_2)
#         if indices[i] != num_features_1
#     ])
#     print("Feature locations shape for First image : ", locations_1_to_use.shape)
#     print("Feature locations shape for Second image : ", locations_2_to_use.shape)
    
# #     print(locations_1_to_use)
# #     print(locations_2_to_use)

#     # Perform geometric verification using RANSAC.
#     try:
#         _, inliers = ransac(
#             (locations_1_to_use, locations_2_to_use),
#             AffineTransform,
#             min_samples=2,
#             residual_threshold=20,
#             max_trials=1000)
#         sum_inliners = sum(inliers)
#         print('Found %d inliers' % sum_inliners)

#         # Visualize correspondences.
#         fig, ax = plt.subplots()
#         inlier_idxs = np.nonzero(inliers)[0]
#         print(inlier_idxs)
#         plot_matches(
#             ax,
#             image1,
#             image2,
#             locations_1_to_use,
#             locations_2_to_use,
#             np.column_stack((inlier_idxs, inlier_idxs)),
#             matches_color='b')
#         ax.axis('off')
#         ax.set_title('DELF correspondences')
#         plt.show()
#         plt.close()
#     except:
#         print("\n Images : " + str(i) + " and " + str(j) + " do not match at all \n\n")
#         sum_inliners = 0
#     return sum_inliners
        

In [ ]:
# for i in range(0, len(images)):
#     for j in range(0, len(images)):
#         if i!=j:
#             match_images(images[i], images[j], results[i], results[j], i , j)

In [ ]:
# for j in range(0, len(images)):
#     match_images(images[0], images[j], results[0], results[j], 0 , j)

#### Below method returns the total number of inliers and total number of common descriptors between two images

In [ ]:
# #@title TensorFlow is not needed for this post-processing and visualization
# def match_images1(image1, image2, result1, result2, i, j):
#     distance_threshold = 0.8

#     # Read features.
#     num_features_1 = result1['locations'].shape[0]

#     num_features_2 = result2['locations'].shape[0]

#     # Find nearest-neighbor matches using a KD tree.
#     d1_tree = cKDTree(result1['descriptors'])
#     _, indices = d1_tree.query(
#         result2['descriptors'],
#         distance_upper_bound=distance_threshold)
# #     print(indices)
#     total_common_unique_descriptors = len(set(indices))
# #     print("\nTotal common descriptors between " + str(i) + " and " + str(j) + " images: ", total_common_unique_descriptors)

#     # Select feature locations for putative matches.
#     locations_2_to_use = np.array([
#         result2['locations'][i,]
#         for i in range(num_features_2)
#         if indices[i] != num_features_1
#     ])
    
#     locations_1_to_use = np.array([
#         result1['locations'][indices[i],]
#         for i in range(num_features_2)
#         if indices[i] != num_features_1
#     ])

#     # Perform geometric verification using RANSAC.
#     try:
#         _, inliers = ransac(
#             (locations_1_to_use, locations_2_to_use),
#             AffineTransform,
#             min_samples=2,
#             residual_threshold=20,
#             max_trials=1000)
#         sum_inliers = sum(inliers)
        
        
#         # Visualize correspondences.
# #         fig, ax = plt.subplots()
# #         inlier_idxs = np.nonzero(inliers)[0]
# # #         print(inlier_idxs)
# #         plot_matches(
# #             ax,
# #             image1,
# #             image2,
# #             locations_1_to_use,
# #             locations_2_to_use,
# #             np.column_stack((inlier_idxs, inlier_idxs)),
# #             matches_color='b')
# #         ax.axis('off')
# #         ax.set_title('DELF correspondences')
# #         plt.show()
# #         plt.close()
#     except:
#         pass
# #         print("Images : " + str(i) + " and " + str(j) + " do not match at all")
#         sum_inliers = 0
# #     print('Found %d inliers' % sum_inliers)
#     return total_common_unique_descriptors, sum_inliers
        

In [ ]:
# comparitive_inliers = 10 
# for i in range(0, len(images)):
#     count = 0
#     max_inliers = 0
#     max_descriptors = 0
#     print("Comparing Image index : ", i)
#     plt.imshow(images[i])
#     plt.show()
#     plt.close()
#     save_j_index_for_max = 0
#     max_inlier_idx = []
#     locations1_max = []
#     locations2_max = []
#     for j in range(0, len(images)):
#         if j != i:
#             total_common_descriptors, inliers = match_images1(images[i], images[j], results[i], results[j], i , j)
#             if inliers > comparitive_inliers:
#                 count += 1
# #             print("\nCommon Descriptors : ", total_common_descriptors)
# #             print("Inliers : ", inliers)
# #             print("\n--------------------------------------------------------------------------------")

#             if max_descriptors < total_common_descriptors:
#                 max_descriptors = total_common_descriptors

#             if max_inliers < inliers:
#                 max_inliers = inliers
#                 save_j_index_for_max = j
                
# #     print("Images " + str(i) + " and " + str(j))
#     plt.imshow(images[save_j_index_for_max])
#     plt.show()
#     plt.close()
#     print("Max number of inliers : ", max_inliers)
#     print("Max number of descriptors : ", max_descriptors)
#     print("Total inliers greater than " + str(comparitive_inliers) + " : ", count)
#     print("\n--------------------------------------------------------------------------------")

In [ ]:
# #@title TensorFlow is not needed for this post-processing and visualization
# def match_images2(image1, image2, result1, result2, i, j):
#     distance_threshold = 0.8

#     # Read features.
#     num_features_1 = result1['locations'].shape[0]

#     num_features_2 = result2['locations'].shape[0]

#     cKDTree_start_time = time.time()
#     # Find nearest-neighbor matches using a KD tree.
#     d1_tree = cKDTree(result1['descriptors'])
#     _, indices = d1_tree.query(
#         result2['descriptors'],
#         distance_upper_bound=distance_threshold)
# #     print("Time to run cKDTree for 1 image : ", time.time() - cKDTree_start_time)
# #     print(indices)
#     total_common_unique_descriptors = len(set(indices))
# #     print("\nTotal common descriptors between " + str(i) + " and " + str(j) + " images: ", total_common_unique_descriptors)

#     # Select feature locations for putative matches.
#     locations_2_to_use = np.array([
#         result2['locations'][i,]
#         for i in range(num_features_2)
#         if indices[i] != num_features_1
#     ])
    
#     locations_1_to_use = np.array([
#         result1['locations'][indices[i],]
#         for i in range(num_features_2)
#         if indices[i] != num_features_1
#     ])
    
#     inlier_idxs = []

#     # Perform geometric verification using RANSAC.
#     try:
#         ransac_start_time = time.time()
#         _, inliers = ransac(
#             (locations_1_to_use, locations_2_to_use),
#             AffineTransform,
#             min_samples=3,
#             residual_threshold=20,
#             max_trials=1000)
# #         print("Time to run ransac for 1 image : ", time.time() - ransac_start_time)
#         sum_inliers = sum(inliers)
#         inlier_idxs = np.nonzero(inliers)[0]
#     except:
#         sum_inliers = 0
#     return total_common_unique_descriptors, sum_inliers, inlier_idxs, locations_1_to_use, locations_2_to_use
        

In [ ]:
# comparitive_inliers = 10
# counts_of_max = 0
# time_to_compare_all_images_in_class = time.time()
# for i in range(0, len(images)):
#     count = 0
#     max_inliers = 0
#     max_descriptors = 0
#     print("Comparing Image index : ", i)
# #     plt.imshow(images[i])
# #     plt.show()
# #     plt.close()
#     save_j_index_for_max = 0
#     inlier_idx_max = []
#     locations1_max = []
#     locations2_max = []
#     class_comparison_start_time = time.time()
#     for j in range(0, len(images)):
#         if j != i:
#             total_common_descriptors, inliers, inlier_idxs, locations1, locations2 = match_images2(images[i], images[j], results[i], results[j], i , j)
#             if inliers > comparitive_inliers:
#                 count += 1
# #             print("\nCommon Descriptors : ", total_common_descriptors)
# #             print("Inliers : ", inliers)
# #             print("\n--------------------------------------------------------------------------------")

#             if max_descriptors < total_common_descriptors:
#                 max_descriptors = total_common_descriptors

#             if max_inliers < inliers:
#                 max_inliers = inliers
#                 save_j_index_for_max = j
#                 inlier_idx_max = inlier_idxs
#                 locations1_max = locations1
#                 locations2_max = locations2
            
#             if inliers > 10:
#                 break
#     print("Time to compare 1 image against all the images in the class : ", time.time()-class_comparison_start_time)
                
# #     print("Images " + str(i) + " and " + str(j))
# #     plt.imshow(images[save_j_index_for_max])
# #     plt.show()
# #     plt.close()
    
#     print("Max number of inliers : ", max_inliers)
#     print("Max number of common descriptors : ", max_descriptors)
#     print("Total inliers greater than " + str(comparitive_inliers) + " : ", count)
    
# #     If max inliers for an image is <= 10 then add the image to dict to filter out later.
# #     otherwise add the landmark_id and image pairs with inliers to another dict to select top 100 pairs later for modeling 
#     if max_inliers <= comparitive_inliers:
#         images_to_filter_out_after_delf[images_paths[i]] = max_inliers
#     else:
# # Actually this is incorrect. SInce we are breaking the loop the moment we see the inliers >10. We are never finding the max unless it is <= 10
#         tuple_list.append((images_paths[i], images_paths[save_j_index_for_max], max_inliers))
#         top_pairs_landmark_id_dict[landmark_id] = (tuple_list)
    
#     try:
#         fig, ax = plt.subplots()
#         plot_matches(
#             ax,
#             images[i],
#             images[save_j_index_for_max],
#             locations1_max,
#             locations2_max,
#             np.column_stack((inlier_idx_max, inlier_idx_max)),
#             matches_color='b')
#         ax.axis('off')
#         ax.set_title('DELF correspondences')
#         plt.show()
#         plt.close()
#     except:
#         pass
#     print("\n--------------------------------------------------------------------------------")
# print("Total time to compare all the images in the class : ", time.time() - time_to_compare_all_images_in_class)

In [21]:
# #@title TensorFlow is not needed for this post-processing and visualization
# def match_images3(image1, image2, result1, result2, i, j):
#     distance_threshold = 0.8

#     # Read features.
#     print(result1)
#     print(result1['locations'].shape[0])
#     num_features_1 = result1['locations'].shape[0]

#     num_features_2 = result2['locations'].shape[0]

#     # Find nearest-neighbor matches using a KD tree.
#     d1_tree = cKDTree(result1['descriptors'])
#     _, indices = d1_tree.query(
#         result2['descriptors'],
#         distance_upper_bound=distance_threshold)

#     # Select feature locations for putative matches.
#     locations_2_to_use = np.array([
#         result2['locations'][i,]
#         for i in range(num_features_2)
#         if indices[i] != num_features_1
#     ])
    
#     locations_1_to_use = np.array([
#         result1['locations'][indices[i],]
#         for i in range(num_features_2)
#         if indices[i] != num_features_1
#     ])

#     # Perform geometric verification using RANSAC.
#     try:
#         _, inliers = ransac(
#             (locations_1_to_use, locations_2_to_use),
#             AffineTransform,
#             min_samples=3,
#             residual_threshold=20,
#             max_trials=1000)

#         sum_inliers = sum(inliers)
#     except:
#         sum_inliers = 0
#     return sum_inliers
        

In [22]:
# max_inliers_reached = 10

# time_to_compare_all_images_in_class = time.time()
# for i in range(0, len(images)):
#     max_inliers = 0
    
#     for j in range(0, len(images)):
#         if j != i:
#             inliers = match_images3(images[i], images[j], results[i], results[j], i , j)

#             if max_inliers < inliers:
#                 max_inliers = inliers
            
#             if inliers > max_inliers_reached:
#                 break
    
# #     If max inliers for an image is <= 10 then add the image to dict to filter out later.
#     if max_inliers <= comparitive_inliers:
#         images_to_filter_out_after_delf[images_paths[i]] = max_inliers
        
# print("Total time to compare all " + str(len(images_paths)) + " images in the landmark_id " + landmark_id + " : ", time.time() - time_to_compare_all_images_in_class)

In [19]:
print(type(images))
print(len(images))
print(images[:2])
print(type(images[0]))
print(images[0])
print(type(images_paths))
print(images_paths[:2])
print(len(images_paths))
print(len(results))

<class 'list'>
26
[<PIL.Image.Image image mode=RGB size=256x256 at 0x24288F51408>, <PIL.Image.Image image mode=RGB size=256x256 at 0x24288F51A88>]
<class 'PIL.Image.Image'>
<PIL.Image.Image image mode=RGB size=256x256 at 0x24288F51408>
<class 'list'>
['datasets\\raw_data\\train\\0\\0052fab157ef0378.jpg', 'datasets\\raw_data\\train\\0\\09ed156ffac3e70c.jpg']
26
26


In [ ]:
#@title TensorFlow is not needed for this post-processing and visualization
def match_images3(multiprocessing_results_list_item):
    result1 = multiprocessing_results_list_item[0]
    
    list_results = multiprocessing_results_list_item[1]
    max_inliers_reached = 10
    max_inlier = 0
    for j in range(len(list_results)):
        result2 = list_results[j]
        
        distance_threshold = 0.8
        # Read features.
        num_features_1 = result1['locations'].shape[0]
        num_features_2 = result2['locations'].shape[0]

        # Find nearest-neighbor matches using a KD tree.
        d1_tree = cKDTree(result1['descriptors'])
        _, indices = d1_tree.query(
            result2['descriptors'],
            distance_upper_bound=distance_threshold)

        # Select feature locations for putative matches.
        locations_2_to_use = np.array([
            result2['locations'][i,]
            for i in range(num_features_2)
            if indices[i] != num_features_1
        ])

        locations_1_to_use = np.array([
            result1['locations'][indices[i],]
            for i in range(num_features_2)
            if indices[i] != num_features_1
        ])

        # Perform geometric verification using RANSAC.
        try:
            _, inliers = ransac(
                (locations_1_to_use, locations_2_to_use),
                AffineTransform,
                min_samples=3,
                residual_threshold=20,
                max_trials=1000)

            sum_inliers = sum(inliers)
        except:
            sum_inliers = 0
            
        if num_features_1 != sum_inliers:
            if max_inlier < sum_inliers:
                max_inlier = sum_inliers
        
            if sum_inliers > max_inliers_reached:
                break
        
            

        

In [32]:
# the multiprocessing_results_list is of type list of tuples, each tuple has position 0 as dictionary, position 1 as list of dictionaries
multiprocessing_results_list = []
for i in range(0, len(results)):
    multiprocessing_results_list.append((results[i], results))

In [34]:
print(type(multiprocessing_results_list))
print(type(multiprocessing_results_list[0]))
print(type(multiprocessing_results_list[0][0]))
print(type(multiprocessing_results_list[0][1]))
print(len(multiprocessing_results_list))
print(multiprocessing_results_list[0][0]['locations'])
print("\n\n\n\n\n\n\n\n\n\n")
print(multiprocessing_results_list[0][1])

<class 'list'>
<class 'tuple'>
<class 'dict'>
<class 'list'>


IndexError: tuple index out of range

In [24]:
# Trying with multiprocessing

time_to_compare_all_images_in_class = time.time()

if __name__ == '__main__':
    max_inliers_reached = 10

    images_to_filter_out = {}
    for i in range(0, len(results)):
        max_inliers = 0
        p = Pool(8)
        inliers = p.map(match_images3, (multiprocessing_results_list))
        p.close()
        p.join()
    #     If max inliers for an image is <= 10 then add the image to dict to filter out later.
#         if max_inliers <= max_inliers_reached:
        images_to_filter_out[images_paths[i]] = inliers
    return images_to_filter_out

# images_to_filter_out_after_delf.update(delf_compare_images_multiprocessing.compare_images(images_paths, images, results))
# if __name__ == '__main__':
    
#     p = Pool(8)
#     temp_dict = p.map(delf_compare_images_multiprocessing.compare_images, (images_paths, results))
#     p.close()
#     p.join()
    
print("Total time to compare all " + str(len(images_paths)) + " images in the landmark_id " + landmark_id + " : ", time.time() - time_to_compare_all_images_in_class)

26
26
Total time to compare all 26 images in the landmark_id 0 :  0.0


In [24]:
images_to_filter_out_after_delf.update(temp_dict[0])

In [ ]:
print(temp_dict)
print(images_to_filter_out_after_delf)

{'datasets\\raw_data\\train\\0\\0ddc92c6353c019a.jpg': 0, 'datasets\\raw_data\\train\\0\\2c8acb7c9958da08.jpg': 7, 'datasets\\raw_data\\train\\0\\b5d8eb502833a58c.jpg': 5}

In [ ]:
print(top_pairs_landmark_id_dict)
print(top_pairs_landmark_id_dict.keys())

In [ ]:
# Multiprocessing not working because of tensorflow imbedded code. some error about tensors
# results = []
# if __name__ == '__main__':
#     start_time_pool = time.time()
#     p = Pool(1)
#     results = p.map(run_delf.run_delf, [[images, delf]])
#     p.close()
#     p.join()
#     print("Total time to run delf for " + str(len(images_paths)) + " images is : ", time.time() - start_time_pool)